In [5]:
import sys
import torch
from torch import nn
import os

sys.path.append(os.path.abspath("../../data"))

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 1

In [6]:
from regression_data import WeatherDatasetWrapper

class WeatherDataset(WeatherDatasetWrapper):
    predictors = ["tmax", "tmin", "rain"]
    target = "tmax_tomorrow"
    sequence_length = 7

wrapper = WeatherDataset(DEVICE)
datasets = wrapper.generate_datasets(BATCH_SIZE)

In [7]:
train = datasets["train"]
valid = datasets["validation"]
test = datasets["test"]

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.rnn = nn.RNN(3, 4, 1, batch_first=True)
        self.dense = nn.Linear(4, 1)

    def forward(self, x):
        x, hidden = self.rnn(x)
        x = self.dense(x)
        return x

In [10]:
from statistics import mean

model = NeuralNetwork().to(DEVICE)
loss_fn = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-2)

EPOCHS = 50
for epoch in range(EPOCHS):
    for batch, (sequence, target) in enumerate(train):
        optimizer.zero_grad()

        sequence = sequence.to(DEVICE)
        pred = model(sequence)

        loss = loss_fn(pred, target)
        loss.backward()
        optimizer.step()

    losses = []
    with torch.no_grad():
        for batch, (sequence, target) in enumerate(valid):
            sequence = sequence.to(DEVICE)
            pred = model(sequence)
            loss = loss_fn(pred, target)
            losses.append(loss.item())

    print(f"Epoch {epoch} valid loss: {mean(losses)}")

Epoch 0 valid loss: 23.59279061642066
Epoch 1 valid loss: 23.244894958504833
Epoch 2 valid loss: 23.5013160391749
Epoch 3 valid loss: 23.834253055143616


KeyboardInterrupt: 